## Teleportation experiment from paper by Fedortchenko

This notebook uses Qubiter to illustrate a pedagogical Teleportation 
experiment performed by S. Fedortchencko on IBM Quantum Experience, 
and described by him in the paper

https://arxiv.org/abs/1607.02398

In [1]:
from SEO_writer import *
from SEO_simulator import *
import numpy as np

Number of qubits is 3.
Note that we use "bit" for both qbits and cbits

In [2]:
num_bits = 3

Use a trivial circuit embedder that embeds 3 qubits into same 3 qubits

In [3]:
emb = CktEmbedder(num_bits, num_bits)

Open a writer, tell it where to write to.
We will use zero bit last (ZL) convention

In [4]:
file_prefix = 'io_folder/teleportation-fedor'
wr = SEO_writer(file_prefix, emb)

Write patial circuit that creates
a state at qbit 0 that will be teleported to qbit 3.

In [5]:
z_axis = 3
def init_ckt():
    wr.write_one_bit_gate(0, OneBitGates.had2)  # H(0)
    wr.write_one_bit_gate(0, OneBitGates.rot_ax, [-np.pi/8, z_axis])  # T(0)
    wr.write_one_bit_gate(0, OneBitGates.had2)  # H(0)
    wr.write_one_bit_gate(0, OneBitGates.rot_ax, [-np.pi/4, z_axis])  # S(0)
init_ckt()
wr.close_files()

init_st_vec = SEO_simulator.get_standard_basis_st([0, 0, 0])
sim = SEO_simulator(file_prefix, num_bits, init_st_vec)
sim.describe_fin_st(print_st_vec=True)

Number of lines in file = 4

Number of Elem. Ops = 4

final state vector
[[[ 0.65328148-0.65328148j  0.00000000+0.j        ]
  [ 0.00000000+0.j          0.00000000+0.j        ]]

 [[ 0.27059805-0.27059805j  0.00000000+0.j        ]
  [ 0.00000000+0.j          0.00000000+0.j        ]]]
total probability of final state vector (=one if no measurements)= 1.0
dictionary with key=qubit, value=final (P(0), P(1))
{0: (0.85355339059327373, 0.14644660940672627), 1: (1.0, 0.0), 2: (1.0, 0.0)}


Now run the whole circuit.
If you want to see the state vector after only first n gates,
where n is less than total number of gates, just
comment out all gates from n+1 to the final one. You can comment
multiple lines with control-slash applied to a selected block. 
control-slash also uncomments a selected, commented block.

In [6]:
wr = SEO_writer(file_prefix, emb)
init_ckt()
wr.write_one_bit_gate(2, OneBitGates.had2)  # H(2)

control_pos = 2
target_pos = 1
trols = Controls.new_knob(num_bits, control_pos, kind=True)
wr.write_controlled_one_bit_gate(target_pos, trols, OneBitGates.sigx)

control_pos = 0
target_pos = 1
trols = Controls.new_knob(num_bits, control_pos, kind=True)
wr.write_controlled_one_bit_gate(target_pos, trols, OneBitGates.sigx)

wr.write_one_bit_gate(0, OneBitGates.had2)  # H(0)

wr.write_measurement(tar_bit_pos=0, kind=0)
wr.write_measurement(tar_bit_pos=1, kind=0)
wr.close_files()

init_st_vec = SEO_simulator.get_standard_basis_st([0, 0, 0])
sim = SEO_simulator(file_prefix, num_bits, init_st_vec)
sim.describe_fin_st(print_st_vec=True)


Number of lines in file = 10

Number of Elem. Ops = 10

final state vector
[[[ 0.32664074-0.32664074j  0.13529903-0.13529903j]
  [ 0.00000000+0.j          0.00000000+0.j        ]]

 [[ 0.00000000+0.j          0.00000000+0.j        ]
  [ 0.00000000+0.j          0.00000000+0.j        ]]]
total probability of final state vector (=one if no measurements)= 0.25
dictionary with key=qubit, value=final (P(0), P(1))
{0: (1.0, 0.0), 1: (1.0, 0.0), 2: (0.85355339059327384, 0.14644660940672616)}


Look in files
* ```io_folder/teleportation-fedor_3_eng.txt```
* ```io_folder/teleportation-fedor_3_ZLpic.txt```

to see the quantum circuit that was generated

If you do the algebra, the final state vector should be
$$ 
\begin{array}{c}|0\rangle \\ |0\rangle \\ (\frac{cc}{2} |0\rangle + \frac{ss}{2}|1\rangle)\end{array}
+
\begin{array}{c}|0\rangle \\ |1\rangle \\ (\frac{cc}{2} |0\rangle + \frac{ss}{2}|1\rangle)\end{array}
+
\begin{array}{c}|1\rangle \\ |0\rangle \\ (\frac{cc}{2} |0\rangle - \frac{ss}{2}|1\rangle)\end{array}
+
\begin{array}{c}|1\rangle \\ |1\rangle \\ (\frac{-ss}{2} |0\rangle + \frac{cc}{2}|1\rangle)\end{array}
$$

where cc = cos(\pi/8) and ss = sin(\pi/8), and where the 
first, second,
third
row corresponds to qubit 0, 1, 2